# Develop an initial model

In [1]:
# Python modules
import dateutil
from datetime import datetime
import re

# Data science packages
import pandas as pd
import numpy as np

# Scikit Learn utility classes & functions
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import make_scorer, mean_absolute_error

# Scikit Learn models
from sklearn.linear_model import Lasso, ElasticNet, Ridge
from sklearn.ensemble import (AdaBoostRegressor, BaggingRegressor, ExtraTreesRegressor, GradientBoostingRegressor,
                              RandomForestRegressor)
from sklearn.svm import SVR, LinearSVR

# Our own code
from src.data.preprocessing import betting_model_df, team_betting_model_df, cum_team_df
from src.model.metrics import measure_estimators

np.random.seed(42)

/usr/local/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
# Set up data and create cumulative features

model_df = betting_model_df()
team_df = team_betting_model_df(model_df)
cum_df = cum_team_df(team_df)
cum_df

team         oppo_team  score  \
team             year round_number                                              
Adelaide         2010 2                     Adelaide            Sydney   75.0   
                      3                     Adelaide         Melbourne   41.0   
                      4                     Adelaide           Carlton   55.0   
                      5                     Adelaide  Western Bulldogs   72.0   
                      6                     Adelaide     Port Adelaide   74.0   
                      7                     Adelaide          Richmond  104.0   
                      8                     Adelaide   North Melbourne   75.0   
                      9                     Adelaide          Brisbane   93.0   
                      10                    Adelaide          St Kilda   76.0   
                      11                    Adelaide         Fremantle  105.0   
                      12                    Adelaide          Hawthorn   77.0   
                      13                    Adelaide         Melbourne  117.0   
                      14                    Adelaide          Essendon  124.0   
                      15                    Adelaide        West Coast  117.0   
                      16                    Adelaide           Geelong   74.0   
                      17                    Adelaide     Port Adelaide   69.0   
                      18                    Adelaide          Richmond   80.0   
                      19                    Adelaide  Western Bulldogs   53.0   
                      20                    Adelaide          Brisbane  104.0   
                      21                    Adelaide       Collingwood   51.0   
                      22                    Adelaide          St Kilda   65.0   
                 2011 1                     Adelaide          Hawthorn  105.0   
                      3                     Adelaide         Fremantle   54.0   
                      4                     Adelaide     Port Adelaide   66.0   
                      5                     Adelaide           Carlton   79.0   
                      6                     Adelaide          St Kilda   90.0   
                      7                     Adelaide         Melbourne   53.0   
                      8                     Adelaide        Gold Coast  130.0   
                      9                     Adelaide       Collingwood   92.0   
                      10                    Adelaide          Brisbane   61.0   
...                                              ...               ...    ...   
Western Bulldogs 2015 21            Western Bulldogs        West Coast   85.0   
                      22            Western Bulldogs   North Melbourne   96.0   
                      23            Western Bulldogs          Brisbane  122.0   
                      25            Western Bulldogs          Adelaide  102.0   
                 2016 1             Western Bulldogs         Fremantle  103.0   
                      2             Western Bulldogs          St Kilda   93.0   
                      3             Western Bulldogs          Hawthorn   90.0   
                      4             Western Bulldogs           Carlton   85.0   
                      5             Western Bulldogs          Brisbane  120.0   
                      6             Western Bulldogs   North Melbourne   45.0   
                      7             Western Bulldogs          Adelaide  123.0   
                      8             Western Bulldogs         Melbourne  114.0   
                      9             Western Bulldogs               GWS   73.0   
                      10            Western Bulldogs       Collingwood   74.0   
                      11            Western Bulldogs        West Coast   83.0   
                      12            Western Bulldogs     Port Adelaide  100.0   
                      13            Western Bulldogs           Geelong   43.0  

In [3]:
# Set up & split data for models

team_features = pd.get_dummies(cum_df.drop(['score', 'oppo_score'], axis=1))
team_labels = pd.Series(cum_df['score'] - cum_df['oppo_score'], name='score_diff')
data = train_test_split(team_features, team_labels)

In [4]:
# Pass data to models & measure performance

linear = (Lasso(), ElasticNet(), Ridge(), LinearSVR())
measure_estimators(linear, data, model_type='regression')

ensemble = (AdaBoostRegressor(),
            BaggingRegressor(),
            ExtraTreesRegressor(),
            GradientBoostingRegressor(),
            RandomForestRegressor(),
            SVR(kernel='rbf'))
measure_estimators(ensemble, data, model_type='regression')



Lasso
Mean CV accuracy: 0.7188143778635837
Test accuracy: 0.732574679943101

Mean CV negative error score: -28.791164269733315
Test error score: 29.129133698565298


ElasticNet
Mean CV accuracy: 0.7178698877644065
Test accuracy: 0.7297297297297297

Mean CV negative error score: -28.992197233861855
Test error score: 29.583121157136382


Ridge
Mean CV accuracy: 0.7183381927480271
Test accuracy: 0.7311522048364154

Mean CV negative error score: -29.07693136463123
Test error score: 29.486379909946386


LinearSVR
Mean CV accuracy: 0.7202339273451835
Test accuracy: 0.7240398293029872

Mean CV negative error score: -29.284470143886058
Test error score: 29.952657286270316


AdaBoostRegressor
Mean CV accuracy: 0.7207157411264086
Test accuracy: 0.7254623044096729

Mean CV negative error score: -29.46279381196043
Test error score: 30.46100049979439


BaggingRegressor
Mean CV accuracy: 0.6941664509011493
Test accuracy: 0.701280227596017

Mean CV negative error score: -30.72708288773064
Test erro